### Очень опасный момент. Сейчас буду пытаться привести изначальный датасет к виду, как в Dano

считаем необходимые датасеты

In [ ]:
part_of_full_data_with_first_cesd <- read.csv("https://raw.githubusercontent.com/EmilEybatove/Health/main/part_of_full_data.csv")

In [ ]:
df <- read.csv("https://raw.githubusercontent.com/EmilEybatove/Health/main/Health.csv")

Для начала, выделим из датасета Dano ключи ID-номер волны
Сделаем это для того, чтобы вытащить запросами нужные данные из изначального датасета

In [ ]:
keys <- df[, c("hhidpn", "index_wave")]

In [ ]:
colnames(part_of_full_data_with_first_cesd)

[1] "X"        "hhidpn"   "r1cesd"   "r2cesd"   "r3cesd"   "r4cesd"  
 [7] "r5cesd"   "r6cesd"   "r7cesd"   "r8cesd"   "r9cesd"   "r10cesd" 
[13] "r11cesd"  "r1proxy"  "r2proxy"  "r3proxy"  "r4proxy"  "r5proxy" 
[19] "r6proxy"  "r7proxy"  "r8proxy"  "r9proxy"  "r10proxy" "r11proxy"
[25] "hacohort" "racohbyr" "raedyrs"

Теперь напишем очень кривой костыль, который мне не нравится. **Его точно нужно будет менять, если мы хотим работать с ещё какими-то данными**. но, пока, для каждой волны выолним соответствующий запрос

In [ ]:
new_format <- data.frame()

for (i in 1:11){
    IDs <- keys[keys$index_wave == i,]$hhidpn # сохраним ID всех домохозяйств, проходивших опрос в данную волну
    cesd <- paste("r", i, "cesd", sep="") # имя соответствующей колонки cesd
    proxy <- paste("r", i, "proxy", sep="") # имя соответствующей колонки proxy
    subset <- part_of_full_data_with_first_cesd[match(IDs, part_of_full_data_with_first_cesd$hhidpn), c("hhidpn", cesd, proxy, "hacohort", "racohbyr", "raedyrs")] # данные о когортах и cesd этой волны
    subset$index_wave <- rep(i, length(subset$hhidpn))
    names(subset)[2] = "cesd"
    names(subset)[3] = "proxy"

    new_format <- rbind(new_format, subset)
}

Эту магию я позже хочу проверить. Подозрительно просто получилось. Match нужен, ибо этот гений не понимает, что обращения по изменённым индексам и номерам строк - не одно и то же. это как обращение в pandas через loc. Увы(((

Теперь, детишки, давайте переходить к самому долгожданному событию этой минуты. будем мёржить датасеты по паре hhidpn и index_wave

In [ ]:
result <- merge(x=new_format, y=df, by=c("hhidpn", "index_wave"))

А теперь сохраним это чудо в новый файл, который в питоне избавим от ужасных .w, .r и.т.п.

In [ ]:
write.csv(result, "merget_Helth.csv")

Теперь перейдём к следущей части